In [1]:
using Plots

In [2]:
function calculateMenStiffnes(mass)#paraméter az ember tömege
    t₀=54.741 #s
    t₂=59.419 #s
    #periódusidő
    Tₜ=(t₂-t₀)/10 #10 ugrás
    #körfrekvencia
    αₜ=2*pi/Tₜ
    #merevség
    sₜ=mass*αₜ^2;
    return sₜ
end

calculateMenStiffnes (generic function with 1 method)

In [3]:
function calculateShipStiffness(mass,statdef)#paraméter a hajó tömege és merülési mélysége
    s=mass*9.81/statdef
end

calculateShipStiffness (generic function with 1 method)

In [4]:
#sajátkör frekvencia és lengéskép számító függvény
function calculate_freq(m1,m2,s1,s2)
    M=[m1 0;0 m2];
    K=[s1 -s1;-s1 s1+s2];
    tmp1=eigvals(K',M);
    tmp2=eigvecs(K',M);
    alfa1=sqrt(tmp1[1])
    alfa2=sqrt(tmp1[2])
    V1=tmp2[:,1]
    V2=tmp2[:,2]
    return alfa1,alfa2,V1,V2
end

calculate_freq (generic function with 1 method)

In [5]:
#α1=fr[1]
#α2=fr2[2]
#A1=fr[3]
#A4=fr[4]

k1(x,fr)=cos(fr[1]*x)*fr[3][1]#idő, alfa1, A1[1]
k2(x,fr)=sin(fr[1]*x)*fr[3][1]#idő, alfa1, A1[1]
k3(x,fr)=cos(fr[2]*x)*fr[4][1]#idő, alfa2, A2[1]
k4(x,fr)=sin(fr[2]*x)*fr[4][1]#idő, alfa2, A2[1]

k5(x,fr)=-1*sin(fr[1]*x)*fr[3][1]*fr[1]#idő, alfa1, A1[1]
k6(x,fr)=cos(fr[1]*x)*fr[3][1]*fr[1]#idő, alfa1, A1[1]
k7(x,fr)=-1*sin(fr[2]*x)*fr[4][1]*fr[2]#idő, alfa2, A2[1]
k8(x,fr)=cos(fr[2]*x)*fr[4][1]*fr[2]#idő, alfa2, A2[1]

k9(x,fr)=cos(fr[1]*x)*fr[3][2]
k10(x,fr)=sin(fr[1]*x)*fr[3][2]
k11(x,fr)=cos(fr[2]*x)*fr[4][2]
k12(x,fr)=sin(fr[2]*x)*fr[4][2]

k13(x,fr)=-1*sin(fr[1]*x)*fr[3][2]*fr[1]
k14(x,fr)=cos(fr[1]*x)*fr[3][2]*fr[1]
k15(x,fr)=-1*sin(fr[2]*x)*fr[4][2]*fr[2]
k16(x,fr)=cos(fr[2]*x)*fr[4][2]*fr[2]

k16 (generic function with 1 method)

In [6]:
x1_2dof(cons,t,freq,deform)=cons[1]*k1(t,freq)+cons[2]*k2(t,freq)+cons[3]*k3(t,freq)+cons[4]*k4(t,freq)-deform[1]
x1p_2dof(cons,t,freq)=cons[1]*k5(t,freq)+cons[2]*k6(t,freq)+cons[3]*k7(t,freq)+cons[4]*k8(t,freq)
x2_2dof(cons,t,freq,deform)=cons[1]*k9(t,freq)+cons[2]*k10(t,freq)+cons[3]*k11(t,freq)+cons[4]*k12(t,freq)-deform[2]
x2p_2dof(cons,t,freq)=cons[1]*k13(t,freq)+cons[2]*k14(t,freq)+cons[3]*k15(t,freq)+cons[4]*k16(t,freq)

x2p_2dof (generic function with 1 method)

In [7]:
function initial_condition(allapot,def,ft)#egy tömbbel tér vissza
    t=allapot[5];
    K=[k1(t,ft) k2(t,ft) k3(t,ft) k4(t,ft);k5(t,ft) k6(t,ft) k7(t,ft) k8(t,ft);k9(t,ft) k10(t,ft) k11(t,ft) k12(t,ft);k13(t,ft) k14(t,ft) k15(t,ft) k16(t,ft)];
    C=inv(K)*[allapot[1]+def[1];allapot[3];allapot[2]+def[2];allapot[4]];
    return C # C[1]=c11 C[2]=c12 C[3]=c21 C[4]=c22 
    #initial_condition end
end

initial_condition (generic function with 1 method)

In [8]:
function calculate_2DOF(state,def,freq,Δ,ϵ)#paraméterek: kezdő állapot
    delta=-1.0;
    constant=initial_condition(state,def,freq)
    t=state[5];
    i=0;
    while delta<0
        t+=Δ;
        i+=1;
        delta=x1_2dof(constant,t,freq,def)-x2_2dof(constant,t,freq,def);
        #println("$t időben $delta a különbség")
        if i>9223372036854775806
            println("ERROR: 9223372036854775806 ciklus után nincs gyök!")
              break;
            #if end
        end  
    end #while end
    
    time_begin=t-Δ;
    time_end=t;
    delta=1.0;
    lepeskoz=Δ;
    
    while delta>ϵ
    lepeskoz=lepeskoz/2;
    idotabla=collect(time_begin:lepeskoz:time_end);
    kulonbsegek=similar(idotabla,Float64)
    for (ijk,ido) in enumerate(idotabla)
        kulonbsegek[ijk]=abs(x1_2dof(constant,ido,freq,def)-x2_2dof(constant,ido,freq,def))
    end
    delta=minimum(kulonbsegek)   
    t=idotabla[indmin(kulonbsegek)]    
        
    end

    return [x1_2dof(constant,t,freq,def) x2_2dof(constant,t,freq,def) x1p_2dof(constant,t,freq) x2p_2dof(constant,t,freq) t]
end

calculate_2DOF (generic function with 1 method)

In [9]:
k17(x,alfa)=cos(alfa*x)
k18(x,alfa)=sin(alfa*x)
k19(x,alfa)=-sin(alfa*x)*alfa
k20(x,alfa)=cos(alfa*x)*alfa

k20 (generic function with 1 method)

In [10]:
function initial_condition(x2,x2p,t,alfa,def)#egy tömbbel tér vissza
    K=[k17(t,alfa) k18(t,alfa);k19(t,alfa) k20(t,alfa)];
    C=inv(K)*[x2+def;x2p];
    return C # C[1]=c1 C[2]=C2 
end

initial_condition (generic function with 2 methods)

In [11]:
x1_1dof(x10,x1p0,Δt)=x10+x1p0*Δt-0.5*9.81*Δt^2
x1p_1dof(x1p0,Δt)=x1p0-9.81*Δt
x2_1dof(cons,t,def,alf)=cons[1]*k17(t,alf)+cons[2]*k18(t,alf)-def
x2p_1dof(cons,t,alf)=cons[1]*k19(t,alf)+cons[2]*k20(t,alf)

x2p_1dof (generic function with 1 method)

In [12]:
function calculate_1DOF(state1,a22,deform,Δ,ϵ)
    delta=1.0;  
    con=initial_condition(state1[2],state1[4],state1[5],a22,deform);    
    t=state1[5]
    i=0;
    while delta>0
        t+=Δ;
        i+=1; 
        delta=x1_1dof(state1[1],state1[3],t-state1[5])-x2_1dof(con,t,deform,a22);
        #println("$t időben $delta a különbség")
        if i>9223372036854775806
            println("ERROR: 9223372036854775806 ciklus után nincs gyök!")
            break;
            #if end
        end 
    end
    
    time_begin=t-Δ
    time_end=t;
    delta=1.0;
    lepeskoz=Δ
    
    while delta>ϵ
    lepeskoz=lepeskoz/2
    idotabla=collect(time_begin:lepeskoz:time_end)    
    kulonbsegek=similar(idotabla,Float64)
    for (ijk,ido) in enumerate(idotabla)
        kulonbsegek[ijk]=abs(x1_1dof(state1[1],state1[3],ido-state1[5])-x2_1dof(con,ido,deform,a22))
    end
    delta=minimum(kulonbsegek)   
    t=idotabla[indmin(kulonbsegek)]
    end
    
    return [x1_1dof(state1[1],state1[3],t-state1[5]) x2_1dof(con,t,deform,a22) x1p_1dof(state1[3],t-state1[5]) x2p_1dof(con,t,a22) t]
end    

calculate_1DOF (generic function with 1 method)

In [39]:
function calculateSimulation(BeginTime::Float64,PeriodCount::Int64,Δ::Float64,ϵ::Float64)
    #Δ=0.001; #számítások időbeli felbontása
    #ϵ=0.00001; #0 ϵ sugarú környezetébe kell eljuttatni a különbséget
    
    #paraméterek
    m1=50; #[kg]
    m2=55; #[kg]
    yst_hajo=0.1; #[m]
    s1=calculateMenStiffnes(m1);
    s2=calculateShipStiffness(m2,yst_hajo);
    g=9.81 #[m/s²]
    
    #2DOF rsz. időfüggvények paraméterei
    frekTömb=calculate_freq(m1,m2,s1,s2); #[1]=α1, [2]=α2, [3]=A1, [4]=A2
    #statikus deformációk:
    statDef=[((m1+m2)*g/s2)+m1*g/s1 (m1+m2)*g/s2]
    
    #1DOF rsz. időfüggvények paraméterei
    α22=sqrt(s2/m2);  

    #a mozgás kezdeti feltételei
    x0g=0.3; #[m]
    x10=-statDef[1]-x0g #[m]
    x20=-statDef[2] #[m]
    x1p0=0; #[m/s]
    x2p0=0; #[m/s]
    
    #szimuláció kerete:
    state=[x10 x20 x1p0 x2p0 BeginTime];
    #kapcsolási pontok tárolása:
    docTime=zeros(Float64,1,PeriodCount*2+1);
    docTime[1]=BeginTime;
    #kapcsolási pontok meghatározása:
    for j=1:PeriodCount
        state=calculate_2DOF(state,statDef,frekTömb,Δ,ϵ)
        docTime[j*2]=state[5];
        state=calculate_1DOF(state,α22,yst_hajo,Δ,ϵ)
        docTime[1+j*2]=state[5];
    end
    
    DataP=50;
    state_values=zeros(Float64,1+2*PeriodCount*DataP,5);#pipa
    state_values[1,:]=[x10 x20 x1p0 x2p0 BeginTime]#pipa
    for i in 1:PeriodCount
        tmp=linspace(docTime[2*i-1],docTime[2*i],DataP+1)#pipa
        state_values[2+(2*i-2)*DataP:1+(2*i-1)*DataP,5]=tmp[2:end];#pipa
        cond=initial_condition(state_values[1+(2*i-2)*DataP,:],statDef,frekTömb)
        for k in 2+(2*i-2)*DataP:1+(2*i-1)*DataP#pipa
            state_values[k,1]=x1_2dof(cond,state_values[k,5],frekTömb,statDef)#x1_2dof
            state_values[k,2]=x2_2dof(cond,state_values[k,5],frekTömb,statDef)#x2_2dof
            state_values[k,3]=x1p_2dof(cond,state_values[k,5],frekTömb)#x1p_2dof
            state_values[k,4]=x2p_2dof(cond,state_values[k,5],frekTömb)#x2p_2dof
        end
        tmp=linspace(docTime[2*i],docTime[2*i+1],DataP+1)#pipa
        cond2=initial_condition(state_values[1+(2*i-1)*DataP,2],state_values[1+(2*i-1)*DataP,4],state_values[1+(2*i-1)*DataP,5],α22,yst_hajo)
        state_values[2+(2*i-1)*DataP:1+2*i*DataP,5]=tmp[2:end];#pipa
        for j in 2+(2*i-1)*DataP:1+2*i*DataP#pipa
            state_values[j,1]=x1_1dof(state_values[1+(2*i-1)*DataP,1],state_values[1+(2*i-1)*DataP,3],state_values[j,5]-state_values[1+(2*i-1)*DataP,5])
            state_values[j,3]=x1p_1dof(state_values[1+(2*i-1)*DataP,3],state_values[j,5]-state_values[1+(2*i-1)*DataP,5])
            state_values[j,2]=x2_1dof(cond2,state_values[j,5],yst_hajo,α22)
            state_values[j,4]=x2p_1dof(cond2,state_values[j,5],α22)
        end
    end
    x1max=maximum(state_values[1:end,1])
    x1pmax=maximum(state_values[1:end,3])
    x2max=maximum(state_values[1:end,2])
    x2pmax=maximum(state_values[1:end,4])
    println("x1max=$x1max")
    println("x2max=$x2max")
    println("x1pmax=$x1pmax")
    println("x2pmax=$x2pmax")
return docTime,state_values
end

calculateSimulation (generic function with 2 methods)

, Float64) in module Main at In[33]:6 overwritten at In[39]:6.


In [50]:
@time eredmeny=calculateSimulation(0.0,200,0.000001,0.00000001);

x1max=0.24132956993789514
x2max=0.2781728486884961
x1pmax=3.5243315810946614
x2pmax=4.530377055922619
 13.385977 seconds (799.82 k allocations: 18.057 MB, 0.04% gc time)


In [15]:
#save("C:/Users/Lajos/Documents/Julia_hardfiles/eredmeny9.jld","eredmeny",eredmeny)

In [51]:
allapotok=eredmeny[2];
timeserie=eredmeny[1];

In [52]:
maximum(allapotok[1:end,1])

0.24132956993789514

In [53]:
plt=1;

In [20]:
#plotlyjs()
gr()
#pyplot()

Plots.GRBackend()

In [54]:
plot(allapotok[1:plt:end,5],[allapotok[1:plt:end,1],allapotok[1:plt:end,2]])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 -0.6 
 
 
 -0.4 
 
 
 -0.2 
 
 
 0.0 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,329.012 39.5038,328.972 39.5151,328.852 39.5264,328.651 39.5377,328.37 39.549,328.01 39.5603,327.571 39.5715,327.053 39.5828,326.457 39.5941,325.784 
 39.6054,325.035 39.6167,324.211 39.628,323.313 39.6392,322.341 39.6505,321.298 39.6618,320.184 39.6731,319.001 39.6844,317.751 39.6956,316.435 39.7069,315.055 
 39.7182,313.613 39.7295,312.109 39.7408,310.548 39.7521,308.929 39.7633,307.256 39.7746,305.531 39.7859,303.755 39.7972,301.931 39.8085,300.062 39.8198,298.149 
 39.831,296.196 39.8423,294.203 39.8536,292.175 39.8649,290.114 39.8762,288.022 39.8874,285.901 39.8987,283.755 39.91,281.586 39.9213,279.397 39.9326,277.19 
 39.9439,274.968 39.9551,272.734 39.9664,270.491 39.9777,268.241 39.989,265.987 40.0003,263.731 40.0115,261.478 40.0228,259.228 40.0341,256.985 40.0454,254.751 
 40.0567,252.529 40.0898,246.086 40.123,239.769 40.1561,233.578 40.1892,227.512 40.2224,221.572 40.2555,215.758 40.2887,210.07 40.3218,204.507 40.3549,199.07 
 40.3881,193.759 40.4212,188.574 40.4544,183.514 40.4875,178.58 40.5206,173.772 40.5538,169.089 40.5869,164.533 40.6201,160.102 40.6532,155.797 40.6864,151.617 
 40.7195,147.564 40.7526,143.636 40.7858,139.834 40.8189,136.158 40.8521,132.607 40.8852,129.182 40.9183,125.883 40.9515,122.71 40.9846,119.662 41.0178,116.74 
 41.0509,113.944 41.084,111.274 41.1172,108.729 41.1503,106.31 41.1835,104.017 41.2166,101.85 41.2497,99.8085 41.2829,97.8927 41.316,96.1027 41.3492,94.4384 
 41.3823,92.8998 41.4154,91.487 41.4486,90.2 41.4817,89.0387 41.5149,88.0032 41.548,87.0934 41.5812,86.3094 41.6143,85.6512 41.6474,85.1186 41.6806,84.7119 
 41.7137,84.4309 41.7295,84.3407 41.7453,84.2753 41.761,84.231 41.7768,84.2043 41.7926,84.1915 41.8083,84.1892 41.8241,84.194 41.8399,84.2026 41.8556,84.212 
 41.8714,84.2189 41.8872,84.2205 41.9029,84.214 41.9187,84.1967 41.9345,84.1662 41.9502,84.1202 41.966,84.0564 41.9818,83.9729 41.9976,83.868 42.0133,83.74 
 42.0291,83.5876 42.0449,83.4095 42.0606,83.2047 42.0764,82.9725 42.0922,82.7123 42.1079,82.4237 42.1237,82.1065 42.1395,81.7609 42.1552,81.3872 42.171,80.9857 
 42.1868,80.5573 42.2026,80.1028 42.2183,79.6235 42.2341,79.1206 42.2499,78.5956 42.2656,78.0504 42.2814,77.4868 42.2972,76.9069 42.3129,76.3131 42.3287,75.7078 
 42.3445,75.0937 42.3602,74.4735 42.376,73.8503 42.3918,73.227 42.4075,72.6069 42.4233,71.9933 42.4391,71.3897 42.4549,70.7996 42.4706,70.2267 42.4864,69.6746 
 42.5022,69.1471 42.541,67.9684 42.5798,66.9622 42.6186,66.1285 42.6575,65.4675 42.6963,64.979 42.7351,64.663 42.7739,64.5197 42.8127,64.5488 42.8516,64.7506 
 42.8904,65.1249 42.9292,65.6718 42.968,66.3912 43.0068,67.2832 43.0457,68.3478 43.0845,69.585 43.1233,70.9947 43.1621,72.5769 43.201,74.3317 43.2398,76.2591 
 43.2786,78.3591 43.3174,80.6316 43.3562,83.0767 43.3951,85.6944 43.4339,88.4846 43.4727,91.4474 43.5115,94.5827 43.5504,97.8906 43.5892,101.371 43.628,105.024 
 43.6668,108.85 43.7056,112.848 43.7445,117.019 43.7833,121.362 43.8221,125.878 43.8609,130.566 43.8998,135.427 43.9386,140.461 43.9774,145.667 44.0162,151.045 
 44.055,156.597 44.0939,162.321 44.1327,168.217 44.1715,174.286 44.2103,180.527 44.2491,186.941 44.288,193.528 44.3268,200.287 44.3656,207.219 44.4044,214.323 
 44.4433,221.6 44.4647,225.685 44.4861,229.803 44.5075,233.936 44.5289,238.062 44.5503,242.163 44.5718,246.22 44.5932,250.212 44.6146,254.121 44.636,257.928 
 44.6574,261.615 44.6788,265.164 44.7002,268.558 44.7217,271.781 44.7431,274.817 44.7645,277.652 44.7859,280.272 44.8073,282.664 44.8287,284.819 44.8502,286.724 
 44.8716,288.373 44.893,289.756 44.9144,290.867 44.9358,291.703 44.9572,292.259 44.9787,292.533 45.0001,292.525 45.0215,292.235 45.0429,291.666 45.0643,290.821 
 45.0857

In [22]:
#savefig("delta60eps90")

In [23]:
    #=    for (j,dl) in enumerate(delta)
        delta[j]=allapotok[j,1]-allapotok[j,2]
    end
    
    
    plot(time_series,delta)
    mod_allapot=copy(allapotok);
        for (j,x1) in enumerate(mod_allapot[:,1])
        mod_allapot[j]+=1.7
        end
=#